In [1]:
import os
import sys
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import SimpleITK as sitk
import skimage
from os.path import expanduser
import glob
from matplotlib.ticker import MaxNLocator
from scipy.stats import spearmanr
import seg_metrics.seg_metrics as sg
import pickle
import matplotlib.patches as mpatches
from skimage import exposure
from scipy import ndimage
import nibabel as nib

In [2]:
print("Python version:", sys.version)
print("Numpy version:", np.__version__)
print("Matplotlib version:", matplotlib.__version__)
print("SITK version:", sitk.__version__)
print("Scikit-image version:", skimage.__version__)

Python version: 3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0]
Numpy version: 1.26.2
Matplotlib version: 3.8.2
SITK version: 2.3.1
Scikit-image version: 0.22.0


In [3]:
patient_id = 20

In [4]:
PREPROCESSED_BASE_DIR = '/mnt/data/datasets/RNSH_HFlung/pre-processed-plastimatch'
PREPROCESSED_MHA_DIR = '{}/mha/Patient{:02d}'.format(PREPROCESSED_BASE_DIR, patient_id)
PREPROCESSED_STACK_DIR = '{}/stack'.format(PREPROCESSED_BASE_DIR)

#### load the Jacobian determinant and calculate the CTVI-Jac

In [5]:
# load images from the Plastimatch (pm) processing
pm_exhale_im = sitk.ReadImage('{}/phase-exhale.mha'.format(PREPROCESSED_MHA_DIR))
pm_inhale_im = sitk.ReadImage('{}/phase-inhale.mha'.format(PREPROCESSED_MHA_DIR))
pm_mask_im = sitk.ReadImage('{}/mask-exhale.mha'.format(PREPROCESSED_MHA_DIR))

pm_exhale_arr = sitk.GetArrayFromImage(pm_exhale_im)
pm_inhale_arr = sitk.GetArrayFromImage(pm_inhale_im)
pm_mask_arr = sitk.GetArrayFromImage(pm_mask_im).astype(bool)

jacobian_det_im = sitk.ReadImage('{}/vf_jac.mha'.format(PREPROCESSED_MHA_DIR))
jacobian_det_arr = sitk.GetArrayFromImage(jacobian_det_im)

ctvi_jac_arr = jacobian_det_arr-1.0
ctvi_jac_arr[~pm_mask_arr] = np.min(ctvi_jac_arr)
ctvi_jac_im = sitk.GetImageFromArray(ctvi_jac_arr)

# set the spacing and origin in the CTVI-Jac image
ctvi_jac_im.SetSpacing(jacobian_det_im.GetSpacing())
ctvi_jac_im.SetOrigin(jacobian_det_im.GetOrigin())

/home/daryl/miniconda3/envs/py310/lib/python3.10/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.3, the latest is 0.5.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [6]:
ctvi_jac_im.GetSize(), ctvi_jac_im.GetSpacing(), ctvi_jac_im.GetOrigin()

((512, 512, 167),
 (0.9765625, 0.9765625, 1.7999999523162842),
 (-249.51171875, -436.01171875, -269.0))

In [7]:
prop = np.sum(np.array(jacobian_det_arr[pm_mask_arr]) < 0) / len(jacobian_det_arr[pm_mask_arr].ravel())
print('proportion of voxels with negative values in Jacobian determinant = {}'.format(round(prop, 2)))

proportion of voxels with negative values in Jacobian determinant = 0.01


#### resample, crop, & pad the CTVI-Jac so we can make a comparison with model predictions

In [8]:
# Resampling the images into (1,1,1)mm voxel size. Can use nearest neighbour for interpolation (default is B-spline). 
def resample_image(itk_image, out_spacing=(1.0, 1.0, 1.0), is_label=False):
    original_spacing = itk_image.GetSpacing()
    original_size = itk_image.GetSize()

    out_size = [int(np.round(original_size[0] * (original_spacing[0] / out_spacing[0]))),
        int(np.round(original_size[1] * (original_spacing[1] / out_spacing[1]))),
        int(np.round(original_size[2] * (original_spacing[2] / out_spacing[2])))]

    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(out_spacing)
    resample.SetSize(out_size)
    resample.SetOutputDirection(itk_image.GetDirection())
    resample.SetOutputOrigin(itk_image.GetOrigin())
    resample.SetTransform(sitk.Transform())
    resample.SetDefaultPixelValue(itk_image.GetPixelIDValue())

    if is_label:
        resample.SetInterpolator(sitk.sitkNearestNeighbor)
    else:
        resample.SetInterpolator(sitk.sitkBSpline)

    return resample.Execute(itk_image)

In [9]:
jacobian_det_im = resample_image(jacobian_det_im)
ctvi_jac_im = resample_image(ctvi_jac_im)

In [10]:
# convert back into numpy array after resampling
jacobian_det_arr = sitk.GetArrayFromImage(jacobian_det_im)
ctvi_jac_arr= sitk.GetArrayFromImage(ctvi_jac_im)

In [11]:
PREPROCESSED_BASE_DIR = '/mnt/data/datasets/RNSH_HFlung/pre-processed-plastimatch'
PREPROCESSED_SHAPING_DIR = '{}/shaping'.format(PREPROCESSED_BASE_DIR)

In [12]:
if not os.path.exists(PREPROCESSED_SHAPING_DIR):
    os.makedirs(PREPROCESSED_SHAPING_DIR)

In [13]:
SHAPING_FILE = '{}/Patient{:02d}.pkl'.format(PREPROCESSED_SHAPING_DIR)

if os.path.isfile(SHAPING_FILE):
    with open(SHAPING_FILE, 'rb') as f:
        shaping_d = pickle.load(f)
else:
    print('could not load the shaping file from {}'.format(SHAPING_FILE))

IndexError: Replacement index 1 out of range for positional args tuple

#### normalise the CTVI-Jac

In [ ]:
normalised_ventilation = exposure.rescale_intensity(ventilation, in_range='image', out_range=(1e-3,1.))

#### apply a median filter

In [ ]:
ventilation_filtered = ndimage.median_filter(normalised_ventilation, size=7)

#### quantise the CTVI-Jac

In [ ]:
percentile_high = np.percentile(ventilation_filtered[mask_exhale],70)
percentile_medium = np.percentile(ventilation_filtered[mask_exhale],30)

In [ ]:
percentile_high,percentile_medium

In [ ]:
high_func_idxs = (ventilation_filtered>percentile_high)
med_func_idxs = (ventilation_filtered<=percentile_high) & (ventilation_filtered>=percentile_medium)
low_func_idxs = (ventilation_filtered<percentile_medium) & (ventilation_filtered>np.min(ventilation_filtered))

ventilation_label_arr = np.full_like(ventilation_filtered, 0)
ventilation_label_arr[high_func_idxs] = 1
ventilation_label_arr[med_func_idxs] = 2
ventilation_label_arr[low_func_idxs] = 3
ventilation_label_arr[~mask_exhale] = 0

#### load the PET ground truth

In [ ]:
output_name = '{}/Patient{:02d}.npy'.format(PREPROCESSED_STACK_DIR, patient_id)
pt_arr = np.load(output_name)
print('loaded stack from {}'.format(output_name))

pet_labels_arr = pt_arr[3]

In [ ]:
pet_labels_arr.shape

#### load the nnU-Net prediction

In [ ]:
# load the predictions for Patient 20
DATASET_ID = 140
TEST_ID = 0

nnunet_prediction_arr = nib.load('/mnt/data/datasets/RNSH_HFlung/nnU-Net-processing/nnUNet_predictions/Dataset{}_RNSH_HFlung/post/RNSH_HFlung_{:04d}.nii.gz'.format(DATASET_ID, TEST_ID)).get_fdata()
nnunet_prediction_arr = np.moveaxis(nnunet_prediction_arr, -1, 0)

In [ ]:
nnunet_prediction_arr.shape

#### visualise the images

In [ ]:
DISPLAY_IDX = 100

fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(1, 5, figsize=(15,4))
ax1.imshow(image_exhale[DISPLAY_IDX], cmap="gray")
ax1.set_title('exhale')
ax1.set_axis_off()

ax2.imshow(image_inhale[DISPLAY_IDX], cmap="gray")
ax2.set_title('inhale')
ax2.set_axis_off()

ax3.imshow(pet_labels_arr[DISPLAY_IDX])
ax3.set_title('PET ground truth')
ax3.set_axis_off()

ax4.imshow(ventilation_label_arr[DISPLAY_IDX])
ax4.set_title('CTVI-Jac')
ax4.set_axis_off()

ax5.imshow(nnunet_prediction_arr[DISPLAY_IDX])
ax5.set_title('CTVI-nnU-Net')
ax5.set_axis_off()

# fig.suptitle('Patient ID {}'.format(patient_id), x=0.1, y=1.0, fontsize=12, ha='left')

fig.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.01, 
                    hspace=0.1)

#### write out the images for processing with EvaluateSegmentation

In [ ]:
HOME_DIR = '/home/daryl'

sitk.WriteImage(sitk.GetImageFromArray(pet_labels_arr), '{}/pet-label.mha'.format(HOME_DIR))
sitk.WriteImage(sitk.GetImageFromArray(ventilation_label_arr), '{}/ctvi-jac-label.mha'.format(HOME_DIR))
sitk.WriteImage(sitk.GetImageFromArray(nnunet_prediction_arr), '{}/ctvi-nnunet.mha'.format(HOME_DIR))

#### read in the results XML and compare segmentation similarity

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
CTVI_JAC_EVAL = '{}/seg-metrics-ctvi-jac.xml'.format(HOME_DIR)
with open(CTVI_JAC_EVAL, 'r') as f:
    data = f.read()
ctvi_jac_eval = BeautifulSoup(data, "xml")

In [ ]:
CTVI_NNUNET_EVAL = '{}/seg-metrics-ctvi-nnunet.xml'.format(HOME_DIR)
with open(CTVI_NNUNET_EVAL, 'r') as f:
    data = f.read()
ctvi_nnunet_eval = BeautifulSoup(data, "xml")

In [ ]:
metrics_l = ['DICE','HDRFDST','MAHLNBS']
print('{: >10}{: >35}{: >15}{: >15}'.format('','','CTVI-Jac','CTVI-nnU-Net'))
for m in metrics_l:
    ctvi_jac_d = ctvi_jac_eval.find(m).attrs
    ctvi_nnunet_d = ctvi_nnunet_eval.find(m).attrs
    print('{: >10}{: >35}{: >15.2f}{: >15.2f}'.format(m, ctvi_jac_d['name'], float(ctvi_jac_d['value']), float(ctvi_nnunet_d['value'])))